
VGG16 Transfer Learning Approach Deep Convolutional Neural networks may take days to train and require lots of computational resources. So to overcome this we will use Transfer Learning for implementing VGG16 with Keras.

Transfer learning is a technique whereby a deep neural network model that was trained earlier on a similar problem is leveraged to create a new model at hand. One or more layers from the already trained model are used in the new model. We will go through more details in a subsequent section below.

Define training and testing path of dataset It is necessary to put the images of both classes of dog and cat in separate subfolders under train and test folder

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
from google.colab.patches import cv2_imshow


Preparation of datasets We generally encountered problems where we try to load a dataset but there is not enough memory in your machine.

Keras provides the ImageDataGenerator class that defines the configuration for image data preparation and augmentation. The generator will progressively load the images in your dataset, allowing you to work with both small and very large datasets containing thousands or millions of images that may not fit into system memory.

In [ ]:
train_path="/content/gdrive/MyDrive/Colab Notebooks/Train"
test_path="/content/gdrive/MyDrive/Colab Notebooks/Validation"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [ ]:

train_path="/content/gdrive/MyDrive/Colab Notebooks/Train"
test_path="/content/gdrive/MyDrive/Colab Notebooks/Validation"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [ ]:

#Sample datasets images
image_dog=cv2.imread("/content/gdrive/MyDrive/Colab Notebooks/Train/dog/dog001.jpg")
cv2_imshow(image_dog)
image_cat=cv2.imread("/content/gdrive/MyDrive/Colab Notebooks/Train/cat/cat001.jpg")
cv2_imshow(image_cat)

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()

In [ ]:

train_generator = train_datagen.flow_from_directory("/content/gdrive/MyDrive/Colab Notebooks/Train",target_size=(224, 224),batch_size=32,shuffle=True,class_mode='binary')
test_generator = test_datagen.flow_from_directory("/content/gdrive/MyDrive/Colab Notebooks/Validation",target_size=(224,224),batch_size=32,shuffle=False,class_mode='binary')

In [ ]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    return model


In [ ]:
model=VGG16()

In [ ]:

model.summary()

In [ ]:
Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)

In [ ]:
Vgg16.load_weights("/content/gdrive/MyDrive/Colab Notebooks/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [ ]:

for layer in Vgg16.layers:
    layer.trainable = False

In [ ]:

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [ ]:
mc = ModelCheckpoint('/content/gdrive/My Drive/best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
model.load_weights("/content/gdrive/My Drive/best_model.h5")

In [ ]:
model.evaluate_generator(test_generator)

In [ ]:
model_json = model.to_json()
with open("/content/gdrive/My Drive/model.json","w") as json_file:
  json_file.write(model_json)


Dogs Vs Cat Classification Inference Program
To recap what we have done till now –

We implemented the VGG16 model with Keras.

We saved the best training weights of the model in a file for future use.

We saved the model in JSON format for reusability.

Now it is time to write an inference program that will do the following –

Load the model that we saved in JSON format earlier.

Load the weight that we saved after training the model earlier.

Compile the model.

Load the image that we want to classify.

Perform classification.

In [ ]:
from keras.models import model_from_json

In [ ]:
def predict_(image_path):
    #Load the Model from Json File
    json_file = open('/content/gdrive/My Drive/model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    #Load the weights
    model_c.load_weights("/content/gdrive/My Drive/best_model.h5")
    #Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    #load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224,224))
    cv2_imshow(image)
    #predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    if preds==0:
        print("Predicted Label:Cat")
    else:
        print("Predicted Label: Dog")

In [ ]:
predict_("/content/gdrive/MyDrive/Colab Notebooks/Train/dog/dog001.jpg")


In [ ]:
predict_("/content/gdrive/MyDrive/Colab Notebooks/Train/cat/cat001.jpg")